In [23]:
import os
import pyart
import matplotlib.pyplot as plt
import shutil
import cartopy.crs as ccrs


# Define the root directory where UF radar data files are located
root_directory = './UF'

# Define the output directory for PNG files
output_directory = './radar_png'

topics = {
    'reflectivity': 'Reflectivity (dBZ)',
    # 'corrected_reflectivity': 'Corrected Reflectivity (dBZ)',
    # 'mean_doppler_velocity': 'Mean Doppler Velocity (m/s)',
    'corrected_velocity': 'Corrected Velocity (m/s)',
    'velocity': 'Velocity (m/s)',
}

In [31]:

def process_uf_file(file_path, topic='reflectivity', title='Reflectivity (dBZ)', output_directory=output_directory, root_directory=root_directory, limit=300, interval=50):
    try:    
        # Create the output directory if it doesn't exist
        os.makedirs(output_directory, exist_ok=True)

        # Set the projection - in this case, we use a general PlateCarree projection
        # projection = ccrs.PlateCarree()     

        # Load the radar data from the UF file
        radar = pyart.io.read(file_path)

        # Create a display object for the radar data
        display = pyart.graph.RadarDisplay(radar)

        # Plot reflectivity data
        fig = plt.figure(figsize=[10, 8])
        ax = fig.add_subplot(111)
        display.plot_ppi(topic, 0, ax=ax, title=title)
        display.set_limits(xlim=(-limit, limit), ylim=(-limit, limit), ax=ax)

        # Display range rings at 50-km intervals
        display.plot_range_rings([num for num in range(interval, limit+1, interval)])

        # Export the plot as an image file to the output directory
        output_filename = os.path.join(output_directory, os.path.basename(file_path).replace('.uf', '.png'))
        plt.savefig(output_filename, dpi=100)
        plt.close()

        print(f'Processed: {file_path}')
    except Exception as e:
        print(f'Error processing {file_path}: {str(e)}')

In [ ]:
# Delete any existing PNG files in the output directory
[shutil.rmtree(f'./{topic}_png') for (topic, title) in topics.items() if os.path.isdir(f'./{topic}_png')]

In [ ]:

# Recursively traverse the directory and its subdirectories
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith('.uf') is False:
            continue

        file_path = os.path.join(root, file)
        [process_uf_file(file_path, output_directory=f'./{topic}_png', topic=topic, title=title) for (topic, title) in topics.items()]

print(f'PNG files exported to: {output_directory}')